In [32]:
import tensorflow as tf
from load_mat import LOAD
import matplotlib.pyplot as plt
import numpy as np

learning_rate = 0.001
iterations = 50000
batch_size = 50
display_step = 1000

channels = 3
image_size = 32
n_classes = 10
dropout = 0.8
hidden = 128
depth_1 = 16
depth_2 = 32
depth_3 = 64
filter_size = 5
normalization_offset = 0.0  # beta
normalization_scale = 1.0  # gamma
normalization_epsilon = 0.001  # epsilon

In [2]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [3]:
def bias_variable(shape):
    return tf.Variable(tf.constant(1.0, shape=shape))

In [4]:
def convolution(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME")

In [5]:
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [8]:
svhn = LOAD("../data/", n_classes, use_extra=True, gray=False)

../data/train_32x32.mat


In [9]:
X = tf.placeholder(tf.float32, [None, image_size, image_size, channels])
y = tf.placeholder(tf.float32, [None, n_classes])

In [10]:
weights = {
    "layer1": weight_variable([filter_size, filter_size, channels, depth_1]),
    "layer2": weight_variable([filter_size, filter_size, depth_1, depth_2]),
    "layer3": weight_variable([filter_size, filter_size, depth_2, depth_3]),
    "layer4": weight_variable([image_size // 8 * image_size // 8 * depth_3, hidden]),
    "layer5": weight_variable([hidden, n_classes])
}

biases = {
    "layer1": bias_variable([depth_1]),
    "layer2": bias_variable([depth_2]),
    "layer3": bias_variable([depth_3]),
    "layer4": bias_variable([hidden]),
    "layer5": bias_variable([n_classes])
}

In [11]:
def normalize(x):
    """ Applies batch normalization """
    mean, variance = tf.nn.moments(x, [1, 2, 3], keep_dims=True)
    return tf.nn.batch_normalization(x, mean, variance, normalization_offset,
                                     normalization_scale,
                                     normalization_epsilon)


In [12]:
def cnn(x):
    # Batch normalization
    x = normalize(x)

    # Convolution 1 -> RELU -> Max Pool
    convolution1 = convolution(x, weights["layer1"])
    relu1 = tf.nn.relu(convolution1 + biases["layer1"])
    maxpool1 = max_pool(relu1)

    # Convolution 2 -> RELU -> Max Pool
    convolution2 = convolution(maxpool1, weights["layer2"])
    relu2 = tf.nn.relu(convolution2 + biases["layer2"])
    maxpool2 = max_pool(relu2)

    # Convolution 3 -> RELU -> Max Pool
    convolution3 = convolution(maxpool2, weights["layer3"])
    relu3 = tf.nn.relu(convolution3 + biases["layer3"])
    maxpool3 = max_pool(relu3)

    # Fully Connected Layer
    shape = maxpool3.get_shape().as_list()
    reshape = tf.reshape(maxpool3, [-1, shape[1] * shape[2] * shape[3]])
    fc = tf.nn.relu(tf.matmul(reshape, weights["layer4"]) + biases["layer4"])

    # Dropout Layer
    keep_prob_constant = tf.placeholder(tf.float32)
    dropout_layer = tf.nn.dropout(fc, keep_prob_constant)

    return (tf.matmul(dropout_layer, weights["layer5"]) + biases["layer5"],
            keep_prob_constant)

In [13]:
y_conv, keep_prob = cnn(X)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,
                                                              logits=y_conv))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
saver = tf.train.Saver()

In [36]:
with tf.Session() as sess:
    # Initialize Tensorflow variables
    sess.run(tf.global_variables_initializer())

    # Variables useful for batch creation
    start = 0
    end = 0

    # The accuracy and loss for every iteration in train and test set
    train_accuracies = []
    train_losses = []
    test_accuracies = []
    test_losses = []

    for i in range(iterations):
        # Construct the batch
        if start == svhn.train_examples:
            start = 0
            end = min(svhn.train_examples, start + batch_size)

        batch_x = svhn.train_data[start:end]
        batch_y = svhn.train_labels[start:end]

        start = end

        # Run the optimizer
        sess.run(optimizer, feed_dict={X: batch_x, y: batch_y,
                                       keep_prob: dropout})

        if (i + 1) % display_step == 0 or i == 0:
            _accuracy, _cost = sess.run([accuracy, cost],
                                        feed_dict={X: batch_x, y: batch_y, keep_prob: 1.0})
            print("Step: {0:6d}, Training Accuracy: {1:5f}, Batch Loss: {2:5f}".format(i + 1, _accuracy, _cost))
            train_accuracies.append(_accuracy)
            train_losses.append(_cost)
            saver.save(sess, 'my-model', global_step=i)

    # Test the model by measuring it's accuracy
    test_iterations = int(round(svhn.test_examples / batch_size + 1))
    for i in range(test_iterations):
        batch_x, batch_y = (svhn.test_data[i * batch_size:(i + 1) * batch_size],
                            svhn.test_labels[i * batch_size:(i + 1) * batch_size])
        _accuracy, _cost = sess.run([accuracy, cost], feed_dict={X: batch_x, y: batch_y, keep_prob: 1.0})
        test_accuracies.append(_accuracy)
        test_losses.append(_cost)
        print("Mean Test Accuracy: {0:5f}, Mean Test Loss: {1:5f}".format(np.mean(test_accuracies), np.mean(test_losses)))



Step:      1, Training Accuracy:   nan, Batch Loss:   nan
Step:   1000, Training Accuracy:   nan, Batch Loss:   nan
Step:   2000, Training Accuracy:   nan, Batch Loss:   nan
Step:   3000, Training Accuracy:   nan, Batch Loss:   nan
Step:   4000, Training Accuracy:   nan, Batch Loss:   nan
Step:   5000, Training Accuracy:   nan, Batch Loss:   nan
Step:   6000, Training Accuracy:   nan, Batch Loss:   nan
Step:   7000, Training Accuracy:   nan, Batch Loss:   nan
Step:   8000, Training Accuracy:   nan, Batch Loss:   nan
Step:   9000, Training Accuracy:   nan, Batch Loss:   nan
Step:  10000, Training Accuracy:   nan, Batch Loss:   nan
Step:  11000, Training Accuracy:   nan, Batch Loss:   nan
Step:  12000, Training Accuracy:   nan, Batch Loss:   nan
Step:  13000, Training Accuracy:   nan, Batch Loss:   nan
Step:  14000, Training Accuracy:   nan, Batch Loss:   nan
Step:  15000, Training Accuracy:   nan, Batch Loss:   nan
Step:  16000, Training Accuracy:   nan, Batch Loss:   nan
Step:  17000, 

Mean Test Accuracy: 0.065474, Mean Test Loss: 18.419950
Mean Test Accuracy: 0.065625, Mean Test Loss: 18.409147
Mean Test Accuracy: 0.066186, Mean Test Loss: 18.397936
Mean Test Accuracy: 0.066122, Mean Test Loss: 18.381821
Mean Test Accuracy: 0.066061, Mean Test Loss: 18.384222
Mean Test Accuracy: 0.066400, Mean Test Loss: 18.346924
Mean Test Accuracy: 0.066337, Mean Test Loss: 18.370361
Mean Test Accuracy: 0.066667, Mean Test Loss: 18.364735
Mean Test Accuracy: 0.066408, Mean Test Loss: 18.371849
Mean Test Accuracy: 0.066731, Mean Test Loss: 18.381327
Mean Test Accuracy: 0.066476, Mean Test Loss: 18.397394
Mean Test Accuracy: 0.066792, Mean Test Loss: 18.406309
Mean Test Accuracy: 0.066916, Mean Test Loss: 18.445631
Mean Test Accuracy: 0.067778, Mean Test Loss: 18.425425
Mean Test Accuracy: 0.067890, Mean Test Loss: 18.417295
Mean Test Accuracy: 0.068000, Mean Test Loss: 18.405664
Mean Test Accuracy: 0.067928, Mean Test Loss: 18.411798
Mean Test Accuracy: 0.068036, Mean Test Loss: 18

Mean Test Accuracy: 0.064096, Mean Test Loss: 18.326469
Mean Test Accuracy: 0.064080, Mean Test Loss: 18.330902
Mean Test Accuracy: 0.064223, Mean Test Loss: 18.316441
Mean Test Accuracy: 0.064206, Mean Test Loss: 18.316807
Mean Test Accuracy: 0.064269, Mean Test Loss: 18.310955
Mean Test Accuracy: 0.064488, Mean Test Loss: 18.309647
Mean Test Accuracy: 0.064392, Mean Test Loss: 18.313906
Mean Test Accuracy: 0.064219, Mean Test Loss: 18.323921
Mean Test Accuracy: 0.064280, Mean Test Loss: 18.322832
Mean Test Accuracy: 0.064186, Mean Test Loss: 18.326611
Mean Test Accuracy: 0.064170, Mean Test Loss: 18.333652
Mean Test Accuracy: 0.064385, Mean Test Loss: 18.326899
Mean Test Accuracy: 0.064368, Mean Test Loss: 18.344091
Mean Test Accuracy: 0.064275, Mean Test Loss: 18.344692
Mean Test Accuracy: 0.064639, Mean Test Loss: 18.334612
Mean Test Accuracy: 0.064848, Mean Test Loss: 18.333630
Mean Test Accuracy: 0.064830, Mean Test Loss: 18.329397
Mean Test Accuracy: 0.064737, Mean Test Loss: 18

Mean Test Accuracy: 0.066883, Mean Test Loss: 18.236677
Mean Test Accuracy: 0.066816, Mean Test Loss: 18.244017
Mean Test Accuracy: 0.066799, Mean Test Loss: 18.241652
Mean Test Accuracy: 0.066832, Mean Test Loss: 18.244717
Mean Test Accuracy: 0.066765, Mean Test Loss: 18.244621
Mean Test Accuracy: 0.066798, Mean Test Loss: 18.237740
Mean Test Accuracy: 0.066880, Mean Test Loss: 18.230745
Mean Test Accuracy: 0.066912, Mean Test Loss: 18.231367
Mean Test Accuracy: 0.067042, Mean Test Loss: 18.229040
Mean Test Accuracy: 0.067122, Mean Test Loss: 18.222357
Mean Test Accuracy: 0.067153, Mean Test Loss: 18.228809
Mean Test Accuracy: 0.067087, Mean Test Loss: 18.231316
Mean Test Accuracy: 0.067215, Mean Test Loss: 18.225758
Mean Test Accuracy: 0.067150, Mean Test Loss: 18.231552
Mean Test Accuracy: 0.067229, Mean Test Loss: 18.235025
Mean Test Accuracy: 0.067308, Mean Test Loss: 18.231155
Mean Test Accuracy: 0.067338, Mean Test Loss: 18.233713
Mean Test Accuracy: 0.067273, Mean Test Loss: 18